# Mục lục
- [Về Guidance](../../../../code/01.Introduce)
- [Cài đặt](../../../../code/01.Introduce)
- [Tạo nội dung không bị ràng buộc](../../../../code/01.Introduce)
- [Phát biểu cho Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Chọn](../../../../code/01.Introduce)
- [Chuỗi suy nghĩ](../../../../code/01.Introduce)
- [Tạo JSON](../../../../code/01.Introduce)
- [Tạo HTML](../../../../code/01.Introduce)


# Về Guidance
Guidance là một thư viện Python mã nguồn mở đã được chứng minh hiệu quả trong việc kiểm soát đầu ra của bất kỳ mô hình ngôn ngữ (LM) nào. Chỉ với một lần gọi API, bạn có thể biểu đạt (bằng Python) các ràng buộc chương trình chính xác mà mô hình phải tuân theo và tạo ra đầu ra có cấu trúc dưới dạng JSON, Python, HTML, SQL, hoặc bất kỳ cấu trúc nào mà trường hợp sử dụng yêu cầu.

Guidance khác biệt so với các kỹ thuật nhắc nhở thông thường. Nó áp đặt các ràng buộc bằng cách điều hướng mô hình từng token một trong lớp suy luận, tạo ra đầu ra chất lượng cao hơn và giảm chi phí cũng như độ trễ tới 30–50% khi sử dụng cho các kịch bản có cấu trúc cao.

Để tìm hiểu thêm về Guidance, hãy truy cập [kho lưu trữ công khai trên GitHub](https://github.com/guidance-ai/guidance) hoặc xem [Phiên thảo luận về Guidance](https://www.youtube.com/watch?v=qXMNPVVlCMs) tại Microsoft Build.


# Cài đặt
1. Cài đặt Guidance bằng `pip install guidance --pre`
2. Triển khai một endpoint Phi 3.5 mini trên Azure bằng cách truy cập https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml và nhấn nút "Deploy"
3. Lưu khóa API của endpoint vào một biến môi trường có tên `AZURE_PHI3_KEY` và URL vào một biến môi trường có tên `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Tạo nội dung không giới hạn
Văn bản có thể được tạo mà không có bất kỳ giới hạn nào bằng cách sử dụng hàm `gen()`. Đây giống như việc sử dụng mô hình mà không có Guidance.

## Định dạng trò chuyện
Giống như nhiều mô hình trò chuyện khác, Phi-3 yêu cầu các tin nhắn giữa người dùng và trợ lý phải tuân theo một định dạng cụ thể. Guidance hỗ trợ mẫu trò chuyện của Phi-3 và sẽ quản lý việc định dạng trò chuyện cho bạn. Để tạo các lượt trò chuyện, hãy đặt từng phần của cuộc hội thoại vào khối `with user()` hoặc `with assistant()`. Khối `with system()` có thể được sử dụng để thiết lập tin nhắn hệ thống.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Tiết kiệm token
Trong các tình huống có cấu trúc cao, Guidance có thể bỏ qua các token và chỉ tạo ra các token cần thiết, cải thiện hiệu suất, tăng hiệu quả và tiết kiệm chi phí API. Các token được tạo ra được hiển thị trong notebook này với nền được làm nổi bật. Các token bắt buộc được hiển thị mà không có làm nổi bật và có chi phí tương đương với các token đầu vào, được ước tính bằng một phần ba chi phí của các token đầu ra.

*Lưu ý:* Ví dụ đầu tiên với việc tạo không bị ràng buộc không thể bắt buộc bất kỳ token nào vì chúng tôi không cung cấp bất kỳ ràng buộc nào.


# Nói về Phi 3  
Với Guidance, bạn có thể dễ dàng chèn văn bản vào phản hồi của mô hình. Điều này có thể hữu ích nếu bạn muốn định hướng đầu ra của mô hình theo một hướng cụ thể.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Giới hạn bằng regex
Trong ví dụ trước, Phi 3 đã trả lời câu hỏi với `Canberra` và sau đó đưa ra các giải thích bổ sung. Để giới hạn đầu ra của mô hình chỉ đúng một từ, có thể sử dụng regex.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# Lựa chọn từ nhiều tùy chọn
Khi một số tùy chọn khả thi đã được biết, bạn có thể sử dụng hàm `select()` để yêu cầu mô hình chọn từ danh sách các tùy chọn.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

Với `select()`, chỉ có token `Can` được tạo ra. Vì `Canberra` là lựa chọn duy nhất có thể hoàn thành phản hồi, các token còn lại đã bị ép buộc.


# Chuỗi Suy Nghĩ
Chuỗi suy nghĩ là một kỹ thuật có thể giúp cải thiện chất lượng đầu ra của mô hình bằng cách khuyến khích nó xử lý vấn đề từng bước một. Thông thường, để đạt được câu trả lời cuối cùng, cần thực hiện nhiều lượt nhắc. Đầu tiên, hướng dẫn mô hình suy nghĩ từng bước. Sau đó, nhắc lại mô hình để cung cấp câu trả lời cuối cùng. Với các API suy luận trò chuyện tiêu chuẩn, điều này cần 2 lần gọi API, và "chuỗi suy nghĩ" mà mô hình tạo ra sẽ bị tính phí hai lần – một lần dưới dạng các token đầu ra khi mô hình tạo ra nó, và một lần nữa dưới dạng các token đầu vào cho lần gọi thứ hai. Với Guidance, toàn bộ quá trình nhiều bước được xử lý và tính phí như một phần của một lần gọi API duy nhất, giảm chi phí và độ trễ.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# Tạo JSON
Guidance có thể được sử dụng để đảm bảo việc tạo JSON tuân thủ theo một JSON schema hoặc mô hình pydantic, chẳng hạn như schema hồ sơ người dùng được hiển thị ở đây.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## Tạo HTML

Hướng dẫn cũng có thể được sử dụng để tạo mã và tuân theo các yêu cầu cú pháp trong ngôn ngữ lập trình. Trong phần này, chúng ta sẽ tạo một chương trình Hướng dẫn nhỏ để viết các trang web HTML đơn giản.

Chúng ta sẽ chia trang web thành các phần nhỏ hơn, mỗi phần có một hàm Hướng dẫn riêng. Sau đó, các phần này được kết hợp trong hàm cuối cùng để tạo ra một trang web HTML.  
Tiếp theo, chúng ta sẽ chạy hàm này trên một mô hình hỗ trợ Hướng dẫn trong Azure AI.

*Lưu ý:* Đây sẽ không phải là một trình tạo HTML đầy đủ tính năng; mục tiêu là để minh họa cách bạn có thể tạo đầu ra có cấu trúc phù hợp với nhu cầu cá nhân của mình.

Chúng ta bắt đầu bằng cách nhập những gì cần thiết từ Hướng dẫn:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

Các trang web HTML có cấu trúc rất chặt chẽ, và chúng ta sẽ 'ép buộc' những phần của trang bằng Guidance.  
Khi chúng ta yêu cầu văn bản từ mô hình một cách rõ ràng, cần đảm bảo rằng nó không bao gồm bất kỳ thứ gì có thể là thẻ - nghĩa là, chúng ta phải loại trừ các ký tự '<' và '>'.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Chúng ta sau đó có thể sử dụng hàm này để tạo văn bản trong một thẻ HTML tùy ý:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Bây giờ, hãy tạo tiêu đề trang.  
Trong phần này, chúng ta cần tạo một tiêu đề trang:  


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

Nội dung của trang HTML sẽ được lấp đầy bằng các tiêu đề và đoạn văn.  
Chúng ta có thể định nghĩa một hàm để thực hiện từng việc:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Bây giờ, chức năng để định nghĩa phần thân của chính HTML.  
Điều này sử dụng `select()` với `recurse=True` để tạo nhiều tiêu đề và đoạn văn:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Tiếp theo, chúng ta đến với hàm tạo ra toàn bộ trang HTML hoàn chỉnh.  
Chúng ta thêm thẻ bắt đầu HTML, sau đó tạo phần tiêu đề, tiếp theo là phần nội dung, và cuối cùng là thêm thẻ kết thúc HTML:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Chúng tôi cung cấp một lớp bao bọc thân thiện với người dùng, cho phép chúng tôi:
- Thiết lập nhiệt độ của quá trình tạo nội dung
- Thu thập trang được tạo từ đối tượng Model


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Chúng ta sau đó có thể ghi đầu ra vào một tệp:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

Và [xem kết quả](../../../../code/01.Introduce/sample_page.html).



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
